# Metrics to calculate:
- % stabilized overall
- % stabilied high severity
- % stabilized low severity
- % stabilized liver
- % stabilized spleen
- % time MAP above safe (map_violations / total episode time * 100%)
- mean time to death (only for episodes ending in death (HR) or failed to advance)
- mean time to stabilization (only for episodes ending in stabilization)
- mean total blood and total cryst per episode (separate high and low severity episodes)
- mean cryst and blood rate (separate high/low sev eps) - fluid rate = total fluid / episode length

In [1]:
import sys
import os

script_dir = script_dir = os.getcwd()
parent_dir = os.path.dirname(script_dir)
sys.path.append(parent_dir)

import pandas as pd
import numpy as np

In [2]:
def calc_bootstrap_outcomes(df, n_boot=1000, ci=95):
    values = df['outcome'].values
    n = len(values)

    boot_stats = [] # list of stabilization rates from each bootstrap sample
    for _ in range(n_boot):
        sample = np.random.choice(values, size=n, replace=True)
        boot_stats.append(np.mean(sample) * 100)

    lower = np.percentile(boot_stats, (100 - ci) / 2)
    upper = np.percentile(boot_stats, 100 - (100 - ci) / 2)

    return np.mean(boot_stats), lower, upper

In [3]:
def calc_stable_stats(df):
    assert(df.iloc[0].id == 0 and df.iloc[-1].id == 49)
    df['outcome'] = (df['outcome'] == "stabilization").astype(int)
    df_highsev = df.iloc[:25, :]
    df_lowsev = df.iloc[25:, :]
    df_liver = df[df['organ']=="liver"]
    df_spleen = df[df['organ']=="spleen"]

    overall = calc_bootstrap_outcomes(df)
    highsev = calc_bootstrap_outcomes(df_highsev)
    lowsev = calc_bootstrap_outcomes(df_lowsev)
    liver = calc_bootstrap_outcomes(df_liver)
    spleen = calc_bootstrap_outcomes(df_spleen)
    return overall, highsev, lowsev, liver, spleen

In [4]:
def calc_bootstrap_time_to_end(df, n_boot=1000, ci=95):
    # ignores truncated episodes
    assert(df.iloc[0].id == 0 and df.iloc[-1].id == 49)
    df_death = df[(df['outcome'] == "failed to advance") | (df['outcome'] == "death (HR)")]
    values = df_death['length'].values
    n = len(values)

    boot_stats_death = []
    for _ in range(n_boot):
        sample = np.random.choice(values, size=n, replace=True)
        boot_stats_death.append(np.mean(sample))
    lower_death = np.percentile(boot_stats_death, (100 - ci) / 2)
    upper_death = np.percentile(boot_stats_death, 100 - (100 - ci) / 2)

    df_stable = df[df['outcome'] == "stabilization"]
    values = df_stable['length'].values
    n = len(values)
    boot_stats_stable = []
    for _ in range(n_boot):
        sample = np.random.choice(values, size=n, replace=True)
        boot_stats_stable.append(np.mean(sample))
    lower_stable = np.percentile(boot_stats_stable, (100 - ci) / 2)
    upper_stable = np.percentile(boot_stats_stable, 100 - (100 - ci) / 2)

    return np.mean(boot_stats_death), lower_death, upper_death, np.mean(boot_stats_stable), lower_stable, upper_stable

In [5]:
# % time MAP above safe (map_violations / total episode time * 100%)
def calc_bootstrap_map_violations(df, n_boot=1000, ci=95):
    assert(df.iloc[0].id == 0 and df.iloc[-1].id == 49)

    values = (df['map_violations'] / df['length']).values * 100
    n = len(values)

    boot_stats = [] # list of % time MAP above safe from each bootstrap sample
    for _ in range(n_boot):
        sample = np.random.choice(values, size=n, replace=True)
        boot_stats.append(np.mean(sample))

    lower = np.percentile(boot_stats, (100 - ci) / 2)
    upper = np.percentile(boot_stats, 100 - (100 - ci) / 2)

    return np.mean(boot_stats), lower, upper

In [6]:
# mean total blood and total cryst per episode (separate high and low severity episodes)
# mean cryst and blood rate (separate high/low sev eps) - fluid rate = total fluid / episode length
def calc_booostrap_fluid_use(df, n_boot=1000, ci=95):
    assert(df.iloc[0].id == 0 and df.iloc[-1].id == 49)

    # high severity
    df_highsev = df.iloc[:25, :]

        # blood
    values_blood_high = df_highsev['blood_total'].values
    n_high = len(values_blood_high)
    boot_stats_blood_high = []
    for _ in range(n_boot):
        sample = np.random.choice(values_blood_high, size=n_high, replace=True)
        boot_stats_blood_high.append(np.mean(sample))
    lower_blood_high = np.percentile(boot_stats_blood_high, (100 - ci) / 2)
    upper_blood_high = np.percentile(boot_stats_blood_high, 100 - (100 - ci) / 2)

        # cryst
    values_cryst_high = df_highsev['cryst_total'].values
    n_high = len(values_cryst_high)
    boot_stats_cryst_high = []
    for _ in range(n_boot):
        sample = np.random.choice(values_cryst_high, size=n_high, replace=True)
        boot_stats_cryst_high.append(np.mean(sample))
    lower_cryst_high = np.percentile(boot_stats_cryst_high, (100 - ci) / 2)
    upper_cryst_high = np.percentile(boot_stats_cryst_high, 100 - (100 - ci) / 2)

    # low severity
    df_lowsev = df.iloc[25:, :]

        # blood
    values_blood_low = df_lowsev['blood_total'].values
    n_low = len(values_blood_low)
    boot_stats_blood_low = []
    for _ in range(n_boot):
        sample = np.random.choice(values_blood_low, size=n_low, replace=True)
        boot_stats_blood_low.append(np.mean(sample))
    lower_blood_low = np.percentile(boot_stats_blood_low, (100 - ci) / 2)
    upper_blood_low = np.percentile(boot_stats_blood_low, 100 - (100 - ci) / 2)

        #  cryst
    values_cryst_low = df_lowsev['cryst_total'].values
    n_low = len(values_cryst_low)
    boot_stats_cryst_low = []
    for _ in range(n_boot):
        sample = np.random.choice(values_cryst_low, size=n_low, replace=True)
        boot_stats_cryst_low.append(np.mean(sample))
    lower_cryst_low = np.percentile(boot_stats_cryst_low, (100 - ci) / 2)
    upper_cryst_low = np.percentile(boot_stats_cryst_low, 100 - (100 - ci) / 2)

    return ((np.mean(boot_stats_blood_high), lower_blood_high, upper_blood_high),
            (np.mean(boot_stats_cryst_high), lower_cryst_high, upper_cryst_high),
            (np.mean(boot_stats_blood_low), lower_blood_low, upper_blood_low),
            (np.mean(boot_stats_cryst_low), lower_cryst_low, upper_cryst_low))

In [7]:
df_moe = pd.read_csv(r'C:\Users\michellexu\Pulse\engine\src\python\pulse\rl-hemorrhage-resuscitation\final_system\eval_data_csvs\eval_data_moe.csv')
df_moe = df_moe.iloc[12:, :]
df_moe.head()

,id,organ,severity,gating,patient,outcome,length,blood_total,cryst_total,vp_total,map_violations
12,0,liver,0.4,tensor([1]),C:\Users\michellexu\Pulse\engine\src\python\pu...,stabilization,21,4021.9482,0.0,0.038430,0
13,1,liver,0.2,tensor([1]),C:\Users\michellexu\Pulse\engine\src\python\pu...,stabilization,21,2417.1455,0.0,0.274458,0
14,2,liver,0.3,tensor([1]),C:\Users\michellexu\Pulse\engine\src\python\pu...,stabilization,21,2937.6519,0.0,0.116922,1
15,3,spleen,1.0,tensor([1]),C:\Users\michellexu\Pulse\engine\src\python\pu...,stabilization,28,2464.3760,0.0,0.016637,3
16,4,liver,0.2,tensor([1]),C:\Users\michellexu\Pulse\engine\src\python\pu...,stabilization,21,2257.0930,0.0,0.281032,0


In [8]:
df_moe.iloc[0].id == 0 and df_moe.iloc[-1].id == 49

np.True_

In [9]:
calc_stable_stats(df_moe)

((np.float64(81.91), np.float64(70.0), np.float64(92.0)),
 (np.float64(63.688), np.float64(44.0), np.float64(84.0)),
 (np.float64(100.0), np.float64(100.0), np.float64(100.0)),
 (np.float64(100.0), np.float64(100.0), np.float64(100.0)),
 (np.float64(72.0875), np.float64(56.25), np.float64(84.375)))

In [37]:
calc_bootstrap_time_to_end(df_moe) # np.mean(boot_stats_death), lower_death, upper_death, np.mean(boot_stats_stable), lower_stable, upper_stable

(np.float64(14.53),
 np.float64(8.0),
 np.float64(18.0),
 np.float64(24.638829268292685),
 np.float64(22.463414634146343),
 np.float64(27.342073170731705))

In [38]:
calc_bootstrap_map_violations(df_moe) # np.mean(boot_stats), lower, upper

(np.float64(6.154010394265233),
 np.float64(1.6900435227854582),
 np.float64(11.385294418842806))

In [44]:
calc_booostrap_fluid_use(df_moe) # ((np.mean(boot_stats_blood_high), lower_blood_high, upper_blood_high),
                                 # (np.mean(boot_stats_cryst_high), lower_cryst_high, upper_cryst_high),
                                 # (np.mean(boot_stats_blood_low), lower_blood_low, upper_blood_low),
                                 # (np.mean(boot_stats_cryst_low), lower_cryst_low, upper_cryst_low))

((np.float64(1980.51081439184),
  np.float64(1517.2358533620002),
  np.float64(2427.9361205399996)),
 (np.float64(1701.49361005192),
  np.float64(692.950285002),
  np.float64(3003.4533563960003)),
 (np.float64(399.84210280524405),
  np.float64(56.5238340229),
  np.float64(782.4059125862)),
 (np.float64(1170.9020148220002),
  np.float64(954.4840458799999),
  np.float64(1397.9714700999998)))

In [63]:
df = pd.read_csv(r'C:\Users\michellexu\Pulse\engine\src\python\pulse\rl-hemorrhage-resuscitation\final_system\eval_data_csvs\eval_data_rppo2.csv')
df = df.iloc[-50:, :]
calc_stable_stats(df)

((np.float64(78.122), np.float64(66.0), np.float64(88.0)),
 (np.float64(83.672), np.float64(68.0), np.float64(96.0)),
 (np.float64(72.092), np.float64(55.90000000000001), np.float64(88.0)),
 (np.float64(83.46666666666667),
  np.float64(66.66666666666666),
  np.float64(100.0)),
 (np.float64(74.978125), np.float64(59.375), np.float64(90.625)))